In [1]:
!pip install numpy scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 6.6 MB/s eta 0:00:0000:0100:01


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import entropy

In [ ]:
# Load the abundance data
csv_path = "/home/dermot.kelly/Dermot_analysis/Phd/Paper_2/rumen_microbiome_pipeline/exported/genus_relative_abundance.csv"
genus_rel_abundance = pd.read_csv(csv_path, index_col=0)

# Extract abundances
abundances = genus_rel_abundance.iloc[0].values

# filter out zeros
abundances = abundances[abundances >0]

# Calculate diversity metrics

# Richness
richness = len(abundances)

